In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from PIL import Image

In [4]:
def count_nobs(station, mode, *filenames):
    def skip_rows(filename): #There is definitely a smarter way to do this, but I was lazy
        df = pd.read_excel(filename)
        n_rows = df.shape[0]
        for i in range(1, n_rows):
            df = pd.read_excel(filename, skiprows=i)
            if df.iloc[0][0] == 'YEAR':
                break
        skiprows = i + 1
        return skiprows

    if station == 'Trinidad Beach & Bay': #Special case: Trinidad Beach & Bay needs to read 2 files
        obs_count = 0
        for filename in filenames:
            skiprows = skip_rows(filename)
            df = pd.read_excel(filename, skiprows=skiprows)
            obs_count += df.shape[0]
    elif station == 'Scripps Pier': #Special case: Scripps Pier has SST and SBT
        skiprows = skip_rows(filenames[0])
        df = pd.read_excel(filenames[0], skiprows=skiprows)
        if mode == 'SST':
            obs_count = df.shape[0]
        elif mode == 'SBT':
            obs_count = np.count_nonzero(~np.isnan(df.BOT_TEMP_C.values))
    else: #normal cases
        skip_rows = skip_rows(filenames[0])
        df = pd.read_excel(filenames[0], skiprows=skip_rows)
        obs_count = df.shape[0]
    return [str(obs_count), station]

In [5]:
farallons = count_nobs('Farallon Islands', 'SST', 'Farallons_TEMP_1925-20230703.xls')
farallons

['35855', 'Farallon Islands']

In [6]:
def draw_nobs(nobs):
    obs_count = int(nobs[0])
    station = nobs[1]
    print(station)
    fig, ax = plt.subplots(figsize=(1,0.1), dpi=400)

    x = np.arange(0, 10, 0.1)
    y = 1

    line, = ax.plot([], [], color='blue', label='Sine wave')

    #plt.xlim([0, 50])
    #plt.ylim([0, 20])

    plt.axis('off')

    text = plt.annotate("", fontsize=85, c='hotpink', xy=(.1, .9), xycoords="axes fraction") 

    def init():
        line.set_data([], [])
        text.set_text('')
        return line, text

    def update(frame):
        if frame <= obs_count:
            text.set_text(str(frame))
        return text,

    ani = FuncAnimation(fig, update, frames=range(obs_count+1), init_func=init, blit=True)
    
    filename = station.replace(' ','')+".gif"
    ani.save(filename, writer='pillow')

    plt.close(fig)

In [8]:
draw_nobs([10, 'test'])

test
